In [39]:
#!g1.1
%pip install rouge_score -q
%pip install transformers -q
%pip install nltk -q
%pip install wandb -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 28.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00


In [41]:
import gdown
import shutil
import wandb

In [ ]:
!wandb login

In [42]:
#!g1.1
# wandb.login(key = 'f91c56848ba54a0f4ee054adfbe86fc8894ebb2a')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
gdown.download('https://drive.google.com/file/d/1jRiENaoNd8bJt0_GCs1JJTMGFlQY-0g_/view?usp=share_link', 'df_processed.pkl', quiet = False, fuzzy = True)

Downloading...
From: https://drive.google.com/uc?id=1jRiENaoNd8bJt0_GCs1JJTMGFlQY-0g_
To: /content/df_processed.pkl
100%|██████████| 101M/101M [00:02<00:00, 43.4MB/s] 


'df_processed.pkl'

In [3]:
gdown.download('https://drive.google.com/file/d/1ubFv7uDAPVAD16vECVCL7cqzXQZclG3_/view?usp=share_link', 'formula_images.zip', quiet = False, fuzzy = True)

Downloading...
From: https://drive.google.com/uc?id=1ubFv7uDAPVAD16vECVCL7cqzXQZclG3_
To: /content/formula_images.zip
100%|██████████| 850M/850M [00:13<00:00, 63.0MB/s]


'formula_images.zip'

In [4]:
gdown.download('https://drive.google.com/file/d/1FqQHsHhDHu3GG0hsla2v-5isDIQGNuVj/view?usp=share_link', 'im2latex_dataset_map.df.pkl.zip', quiet = False, fuzzy = True)

Downloading...
From: https://drive.google.com/uc?id=1FqQHsHhDHu3GG0hsla2v-5isDIQGNuVj
To: /content/im2latex_dataset_map.df.pkl.zip
100%|██████████| 11.9M/11.9M [00:00<00:00, 34.6MB/s]


'im2latex_dataset_map.df.pkl.zip'

In [5]:
shutil.unpack_archive('formula_images.zip')

In [6]:
shutil.unpack_archive('im2latex_dataset_map.df.pkl.zip')

In [13]:
!pip install einops
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.2 MB/s eta 0:00:00


In [49]:
#!g1.1
import numpy as np
import torch

import torch.nn as nn
import torch.nn.functional as F
from einops.layers.torch import Rearrange
from einops import rearrange
import math
import cv2
import os
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

from torchvision.transforms import transforms as T
from torchvision.transforms import functional as F
from transformers import AutoTokenizer
import pandas as pd
from PIL import Image
from rouge_score import rouge_scorer

In [15]:
#!g1.1
tokenizer = AutoTokenizer.from_pretrained('nlpconnect/vit-gpt2-image-captioning')
# tokenizer = AutoTokenizer.from_pretrained('gpt2')


In [16]:
#!g1.1
df = pd.read_pickle('im2latex_dataset_map.df.pkl')
df

,image,formula_name,latex
0,23be72ded29e9b4_basic.png,23be72ded29e9b4_basic,\int _ { - \epsilon } ^ { \infty } d l \: \mat...
1,8c904c5d9c7bd63_basic.png,8c904c5d9c7bd63_basic,d s ^ { 2 } = ( 1 - { \frac { q c o s \theta }...
2,1f111caa1be3516_basic.png,1f111caa1be3516_basic,\widetilde \gamma _ { \mathrm { h o p f } } \s...
3,76287b1b483a0d2_basic.png,76287b1b483a0d2_basic,"( { \cal L } _ { a } g ) _ { i j } = 0 , \ \ \..."
4,7483d06c1e14dcb_basic.png,7483d06c1e14dcb_basic,S _ { s t a t } = 2 \pi \sqrt { N _ { 5 } ^ { ...
...,...,...,...
154939,bdf0210244a0cd3_basic.png,bdf0210244a0cd3_basic,"D _ { g } ^ { g } ( \xi , Y ) | _ { D L } = \b..."
154940,f09875fed00506d_basic.png,f09875fed00506d_basic,\left( \begin{array} { l l } { M _ { \tilde { ...
154941,0d49cda875fc56c_basic.png,0d49cda875fc56c_basic,g ^ { 4 } \int { \frac { d ^ { 4 } p } { ( 2 \...
154942,db23b0939638460_basic.png,db23b0939638460_basic,E ( R ) \! = \! 2 \pi \! \! \int _ { 0 } ^ { \...


In [17]:
#!g1.1
df.latex = df.latex.apply(lambda x: '$' + x + '$')

In [18]:
#!g1.1
df

,image,formula_name,latex
0,23be72ded29e9b4_basic.png,23be72ded29e9b4_basic,$\int _ { - \epsilon } ^ { \infty } d l \: \ma...
1,8c904c5d9c7bd63_basic.png,8c904c5d9c7bd63_basic,$d s ^ { 2 } = ( 1 - { \frac { q c o s \theta ...
2,1f111caa1be3516_basic.png,1f111caa1be3516_basic,$\widetilde \gamma _ { \mathrm { h o p f } } \...
3,76287b1b483a0d2_basic.png,76287b1b483a0d2_basic,"$( { \cal L } _ { a } g ) _ { i j } = 0 , \ \ ..."
4,7483d06c1e14dcb_basic.png,7483d06c1e14dcb_basic,$S _ { s t a t } = 2 \pi \sqrt { N _ { 5 } ^ {...
...,...,...,...
154939,bdf0210244a0cd3_basic.png,bdf0210244a0cd3_basic,"$D _ { g } ^ { g } ( \xi , Y ) | _ { D L } = \..."
154940,f09875fed00506d_basic.png,f09875fed00506d_basic,$\left( \begin{array} { l l } { M _ { \tilde {...
154941,0d49cda875fc56c_basic.png,0d49cda875fc56c_basic,$g ^ { 4 } \int { \frac { d ^ { 4 } p } { ( 2 ...
154942,db23b0939638460_basic.png,db23b0939638460_basic,$E ( R ) \! = \! 2 \pi \! \! \int _ { 0 } ^ { ...


In [19]:
#!g1.1
class CustomDataset(Dataset):
    
    
    def __init__(self, data, tokenizer, transform = None, max_length = 150):
        
        self.data = data 
        
        self.tok = tokenizer
        
        self.transform = transform
        
        self.max_length = max_length
        
    def __len__(self):
        
        return len(self.data)
    
    
    def __getitem__(self, index):
            
        cur_row = self.data.iloc[index]
        
        x = Image.open(os.path.join(f'formula_images/{cur_row.image}'))
        
        y = self.tok(cur_row.latex, return_tensors = 'pt', max_length = self.max_length, 
                           add_special_tokens = True, truncation = True, padding = 'max_length')['input_ids'].squeeze()

#         y = self.tok(cur_row.latex, return_tensors = 'pt')['input_ids'].squeeze()
        
        if self.transform is not None:
            
            x = self.transform(x)
            
        return x, y
        
        
        
        

In [20]:
#!g1.1
max_length = 150
dataset = CustomDataset(df, tokenizer, T.Compose([T.ToTensor(), T.Resize(size = (224, 224))]), max_length = max_length)

In [21]:
#!g1.1

batch_size = 64

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

In [22]:
#!g1.1
from torch.utils.data import Subset

In [23]:
#!g1.1
ids = list(range(len(dataset)))

train_ids = ids[:train_size]
test_ids = ids[train_size:]

train_dataset = Subset(dataset, train_ids)
val_dataset = Subset(dataset, train_ids)

train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

In [74]:
val_dataset = Subset(dataset, test_ids)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

In [73]:
torch.save(train_dataloader, 'train_loader_night.pt')

In [24]:
# #!g1.1
# train_subset = Subset(train_dataset, [1,2,3,4])
# val_subset = Subset(val_dataset, [1,2,34])

In [25]:
# #!g1.1
# train_dataloader = DataLoader(train_subset, batch_size = 1, shuffle = True)
# val_dataloader = DataLoader(val_subset, batch_size = 1, shuffle = False)

In [26]:
# #!g1.1
# tokenizer(df.latex[0], return_tensors = 'pt')['input_ids'].squeeze().shape

In [27]:
#!g1.1
import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

In [28]:
#!g1.1
class AttentionBlock(torch.nn.Module):
    def __init__(self, emb_size , head_dim, dropout = 0.1, is_decoder = False): 
        
        super().__init__()
        self.q = nn.Linear(emb_size, head_dim, bias = False)
        self.k = nn.Linear(emb_size, head_dim, bias = False)
        self.v = nn.Linear(emb_size, head_dim, bias = False)
        self.scale = head_dim ** -0.5
        self.drop = nn.Dropout(dropout)
        self.is_decoder = is_decoder
      
      
    def forward(self, q, k, v):
        
        q = self.q(q)
        k = self.k(k)
        v = self.v(v)
        
        B, T, C = q.shape
        qk = (q @ k.transpose(-2, -1)) * self.scale
        
        if self.is_decoder:
            qk = qk.masked_fill(torch.tril(torch.ones(T, T, device = device)) == 0, float('-inf'))
        qks = F.softmax(qk, dim = -1)
        qks = self.drop(qks)
        att = qks @ v
        
        return att
        
        
        
      
      
    

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        # print(attn_scores.shape)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output
    
    
# class PositionalEncoding(nn.Module):
#     def __init__(self, patch_size, embed_size):
        
#         super().__init__()
#         self.pos = nn.Parameter(torch.randn(1, patch_size ** 2 + 1, embed_size))
        
#     def forward(self, x):
        
#         return x + self.pos

class EncoderLayer(torch.nn.Module):
    
    def __init__(self, dim, n_heads, dropout = 0.1):
        super().__init__()
        
        self.sa = MultiHeadAttention(dim, n_heads)
        
        self.ffn = nn.Sequential(
            nn.Linear(dim, 4*dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(4*dim, dim),
            nn.Dropout(dropout)
        )
        self.ln1 = nn.LayerNorm(dim)
        self.ln2 = nn.LayerNorm(dim)
    
    def forward(self, x):
        
        x = x + self.ln1(self.sa(x,x, x))
        x = x + self.ln2(self.ffn(x))
#         x = x + self.sa(self.ln1(x))
#         x = x + self.ffn(self.ln2(x))
        
        return x
  

class DecoderLayer(torch.nn.Module):
    def __init__(self, d_model, num_heads, dropout = 0.1):
        
        super().__init__()
        
        self.cross_att = MultiHeadAttention(d_model, num_heads)
        self.masked_att = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
        self.ffn = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(4 * d_model, d_model),
            nn.Dropout(dropout)
        )
     
        
    def forward(self, x, enc_output):
        
        N, seq_length, _ = x.shape
        
#         masked_att = self.masked_att(x, x, x, mask = torch.tril(torch.ones(seq_length, seq_length,device=device)))
        masked_att = self.masked_att(x, x, x, mask = torch.tril(torch.ones(seq_length, seq_length)).expand(N, 1, seq_length, seq_length).to(device))
        x = self.norm1(x + self.dropout(masked_att))
        
        cross_mask = self.cross_att(x, enc_output, enc_output)
        x = self.norm2(x + self.dropout(cross_mask))
        
        ff_out = self.ffn(x)
        
        x = self.norm3(x + self.dropout(ff_out))
        
        return x
            
  
class Transformer(torch.nn.Module):
    
    def __init__(self, emd_size, hidden_size, n_heads, n_blocks, max_length, patch_size = 16):
        super().__init__()
        
        
        self.lp_src = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_size, p2 = patch_size),
            nn.LayerNorm(patch_size**2),
            nn.Linear(patch_size**2, emd_size),
            nn.LayerNorm(emd_size),
        )

        self.lp_tgt = nn.Embedding(vocab_size, emd_size)
        
        
        
        self.pos_src = nn.Parameter(torch.randn(1, 14**2, emd_size))
        
        self.pos_tgt = nn.Embedding(max_length, emd_size)
        
        
        self.max_length = max_length

        self.encoder_blocks = nn.Sequential(
            *[EncoderLayer(emd_size, n_heads) for _ in range(n_blocks)]
        )
        
        self.decoder_blocks = nn.Sequential(
            *[DecoderLayer(emd_size, n_heads) for _ in range(n_blocks)]
        )

        self.mlp = nn.Sequential(
            
            nn.Linear(emd_size, hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, vocab_size)
        )
        

        
    def forward(self, src, tgt): #, tgt):
        
        
        src = self.lp_src(src)


        tgt = self.lp_tgt(tgt)

        src = src + self.pos_src

        
        tgt += self.pos_tgt(torch.arange(tgt.shape[1], device=device).expand(tgt.shape[0], tgt.shape[1]))
        
        for block in self.encoder_blocks:
            src = block(src)

          
        for block in self.decoder_blocks:
            tgt = block(tgt, src)
    
        tgt = self.mlp(tgt)
        
        return tgt

        
        

In [29]:
#!g1.1
emd_size = 512
hidden_size = 512
n_heads = 64
n_blocks = 5

vocab_size = len(tokenizer)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Transformer(emd_size, hidden_size, n_heads, n_blocks, max_length).to(device)

In [30]:
#!g1.1
a = torch.randn(size = (1, 1, 224, 224), device = device)
b = torch.randint(0, vocab_size, size = (1, 150),device = device)

In [31]:
#!g1.1
model(a,b).shape

torch.Size([1, 150, 50257])

In [32]:
#!g1.1
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 1e-4)

In [45]:
#!g1.1
model_parameters = {
    'emd_size':emd_size,
    'hidden_size' :hidden_size,
    'n_heads':n_heads,
    'n_blocks':n_blocks,
    'max_length': max_length
}

wandb.init(
    
    project="imgToLatex",
    
    
    config={
        **model_parameters, 'max_length' : max_length, 'lr': 1e-4
    },

    name = f'VIT Encoder Full Dataset Previous Night Run'


)

wandb: Currently logged in as: baluevd (skoltech7). Use `wandb login --relogin` to force relogin


In [101]:
#!g1.1
from rouge_score import rouge_scorer
def compute_metrics(preds, labels, is_test = False):
    metrics = ['precision', 'recall', 'fmeasure']
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
    scores = scorer.score(''.join(pred_str), ''.join(labels_str))
    
    if is_test:
        return {f'{key}_{metrics[idx]}_test': score for key in scores for idx, score in enumerate(scores[key])}
    else:
        return {f'{key}_{metrics[idx]}': score for key in scores for idx, score in enumerate(scores[key])}


@torch.no_grad()
def generate_batch(model, img, is_decode = False):
    
    model.eval()
    img = img.to(device)
    
    N = img.shape[0]
    symbols = torch.tensor([3], device = device).expand(N, 1)
    
    for _ in range(max_length - 1):
        
        cur_symbols = model(img, symbols).argmax(-1)[:, -1][:, None]
        symbols = torch.cat([symbols, cur_symbols], dim = 1)
        
    if is_decode:
        symbols = tokenizer.batch_decode(symbols, skip_special_tokens=True)
    return symbols

In [104]:
# #!g1.1
# from tqdm.auto import tqdm
# def epoch_train(model, criterion, optim, loader, device):
    
#     model.train()
#     n_correct = 0
#     loss_total = 0
#     rang = tqdm(loader)
#     for img, labels in rang:
#         cleanup()
#         N = img.shape[0]
#         img = img.to(device)
#         labels = labels.to(device)
#         outputs = model(img, labels[:,:-1])
# #         print(outputs.shape)
#         loss = criterion(outputs.view(N * (max_length - 1), -1), labels[:,1:].reshape(-1))

#         loss_total+=loss.item()
            
#         optim.zero_grad()
#         loss.backward()
#         optim.step()
        
#         rang.set_description(f'Loss: {loss.item():.3f}')
        
#     return 0, loss_total/(len(loader))

# def epoch_test(model, loader, criterion, device):
    
#     model.eval()
#     n_correct = 0
#     loss_total = 0
#     rang = tqdm(loader)
    
#     for img, labels in rang:
#         cleanup()
#         N = img.shape[0]
#         img = img.to(device)
#         labels = labels.to(device)
#         outputs = model(img, labels[:,:-1])
        
#         loss = criterion(outputs.view(N * (max_length - 1), -1), labels[:,1:].reshape(-1))
        
#         loss_total += loss.item()
        
#         rang.set_description(f'Loss: {loss.item():.3f}')
            
#     return 0, loss_total/(len(loader))

# def train(model, criterion, optim, train_loader, test_loader, epochs, device):
    
#     acc_train_list, loss_train_list = [], []
#     acc_test_list, loss_test_list = [], []
    
#     for epoch in tqdm(range(epochs)):
        
        
#         acc_train, loss_train = epoch_train(model, criterion, optim, train_loader, device)
#         acc_test, loss_test = epoch_test(model, test_loader,criterion, device)
        
#         acc_train_list.append(acc_train)
#         acc_test_list.append(acc_test)
        
#         loss_train_list.append(loss_train)
#         loss_test_list.append(loss_test)
        
        
#         print(f'Epoch {epoch+1}/{epochs}: Acc_train - {acc_train:.3f}, Loss_train - {loss_train:.3f}'+\
#              f' Acc_test - {acc_test:.3f}, Loss_test - {loss_test:.3f}')
#     return acc_train_list, loss_train_list, acc_test_list, loss_test_list


#!g1.1
from tqdm.auto import tqdm
def epoch_train(model, criterion, optim, loader, device):
    
    model.train()
    n_correct = 0
    loss_total = 0
    rang = tqdm(loader)
    for idx, (img, labels) in enumerate(rang):
        cleanup()
        N = img.shape[0]
        img = img.to(device)
        labels = labels.to(device)
        outputs = model(img, labels[:,:-1])

        loss = criterion(outputs.view(N * (max_length - 1), -1), labels[:,1:].reshape(-1))

        loss_total+=loss.item()
        # if idx % 2 == 0:   
        #     optim.zero_grad()
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        rang.set_description(f'Loss: {loss.item():.3f}')
        
        
        if (idx + 1) % (100) == 0:
            metrics = compute_metrics(generate_batch(model, img), labels)
            wandb.log({
                'loss_train': loss.item(),
                **metrics, 
            })
        
    return 0, loss_total/(len(loader))

@torch.no_grad()
def epoch_test(model, loader, criterion, device):
    
    model.eval()
    n_correct = 0
    loss_total = 0
    rang = tqdm(loader)
    
    for idx, (img, labels) in enumerate(rang):
        cleanup()
        N = img.shape[0]
        img = img.to(device)
        labels = labels.to(device)
        outputs = model(img, labels[:,:-1])
        
        loss = criterion(outputs.view(N * (max_length - 1), -1), labels[:,1:].reshape(-1))
        
        loss_total += loss.item()
        
        rang.set_description(f'Loss: {loss.item():.3f}')

        if (idx + 1) % (50) == 0:
            metrics = compute_metrics(generate_batch(model, img), labels, is_test=True)
            wandb.log({
                'loss_test': loss.item(),
                **metrics, 
            })
            
    return 0, loss_total/(len(loader))

def train(model, criterion, optim, train_loader, test_loader, epochs, device):
    
    acc_train_list, loss_train_list = [], []
    acc_test_list, loss_test_list = [], []
    
    for epoch in tqdm(range(epochs)):
        
        
        acc_train, loss_train = epoch_train(model, criterion, optim, train_loader, device)
        acc_test, loss_test = epoch_test(model, test_loader,criterion, device)
        
        acc_train_list.append(acc_train)
        acc_test_list.append(acc_test)
        
        loss_train_list.append(loss_train)
        loss_test_list.append(loss_test)
        
        
#         print(f'Epoch {epoch+1}/{epochs}: Acc_train - {acc_train:.3f}, Loss_train - {loss_train:.3f}'+\
#              f' Acc_test - {acc_test:.3f}, Loss_test - {loss_test:.3f}')
    return acc_train_list, loss_train_list, acc_test_list, loss_test_list

In [105]:
#!g1.1
acc_train_list, loss_train_list, acc_test_list, loss_test_list = train(model, criterion, 
                                                          optim, 
                                                          train_loader = train_dataloader, 
                                                          test_loader = val_dataloader, 
                                                          epochs=20, 
                                                          device = device)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/1937 [00:00<?, ?it/s]

  0%|          | 0/485 [00:00<?, ?it/s]

  0%|          | 0/1937 [00:00<?, ?it/s]

Exception ignored in: <function _xla_gc_callback at 0x7f3c9c75fbe0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: ignored

In [106]:
#!g1.1
model_parameters = {
    'emd_size':emd_size,
    'hidden_size' :hidden_size,
    'n_heads':n_heads,
    'n_blocks':n_blocks,
    'max_length': max_length
}

torch.save(model.state_dict(), 'transformer_night_v2.pt')
torch.save(model_parameters, 'model_parameters_night_v2.pt')

In [ ]:
# #!g1.1
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# m = torch.load('model/transformer1.pt')
# model_par = torch.load('model/model_parameters1.pt')

# model = Transformer(**model_par).to(device)

# model.load_state_dict(m)

<All keys matched successfully>

In [78]:
len(val_dataset)

30989

In [117]:
#!g1.1
model.eval()
img, tar = val_dataset[277]
img = img.to(device)
img = img.unsqueeze(0)
    
symbols = torch.tensor([[3]], device = device)

for i in range(149):
    cur_symb = model(img, symbols).argmax(2)[:,-1].unsqueeze(0)
        
    symbols = torch.cat([symbols, cur_symb], dim = 1)
    
tokenizer.decode(symbols[0].cpu(), skip_special_tokens = True)

'$\\Delta \\Delta \\phi _ { q } ^ { 2 } = \\frac { 1 } { 4 \\pi } \\operatorname { l n } \\frac { 2 \\nu ^ { 2 } } { 3 \\pi ^ { 2 } - ( \\nu + \\Delta ) ( \\Delta q ) ^ { 2 } }.$'

In [118]:
#!g1.1
tokenizer.decode(tar.cpu(), skip_special_tokens = True)

'$\\Delta \\langle \\phi _ { q } ^ { 2 } \\rangle = \\frac { 1 } { 4 \\pi } \\operatorname { l n } \\frac { 2 c ^ { 2 } } { 3 \\phi _ { c } ^ { 2 } - c ^ { 2 } + 3 \\Delta \\langle \\phi _ { q } ^ { 2 } \\rangle }.$'

$\Delta \langle \phi _ { q } ^ { 2 } \rangle = \frac { 1 } { 4 \pi } \operatorname { l n } \frac { 2 c ^ { 2 } } { 3 \phi _ { c } ^ { 2 } - c ^ { 2 } + 3 \Delta \langle \phi _ { q } ^ { 2 } \rangle }.$

$\Delta \Delta \phi _ { q } ^ { 2 } = \frac { 1 } { 4 \pi } \operatorname { l n } \frac { 2 \nu ^ { 2 } } { 3 \pi ^ { 2 } - ( \nu + \Delta ) ( \Delta q ) ^ { 2 } }.$

In [120]:
from google.colab import drive

In [121]:
drive.mount('/content/mydrive/')

Mounted at /content/mydrive/


In [122]:
# #!g1.1
# model_parameters = {
#     'emd_size':emd_size,
#     'hidden_size' :hidden_size,
#     'n_heads':n_heads,
#     'n_blocks':n_blocks,
#     'max_length': max_length
# }

# torch.save(model.state_dict(), '/content/mydrive/MyDrive/transformer_night_v2.pt')
# torch.save(model_parameters, '/content/mydrive/MyDrive/model_parameters_night_v2.pt')

# # '/content/mydrive/MyDrive'

$e x p \left( \frac { \partial } { \partial x _ { 1 } } \frac { \partial } { \partial x _ { 1 } } \right) \frac { \partial } { \partial x _ { 1 } }$

$e x p \left( - \frac { \partial } { \partial \alpha _ { j } } \theta ^ { j k } \frac { \partial } { \partial \alpha _ { k } } \right)$

In [ ]:
#!g1.1

def generate(model, img):
    
    model.eval()
#     cur_symb = torch.tensor([[3]], device = device)
    
    symbols = torch.tensor([[3]], device = device)
    with torch.no_grad():
        
        while (symbols[:,-1].item() != 50256) or (symbols.shape[1] != max_length):

            cur_symb = model(img, symbols).argmax(2)[:,-1].unsqueeze(0)

            symbols = torch.cat([symbols, cur_symb], dim = 1)
        
    return symbols.cpu().detach()
        
        

In [ ]:
#!g1.1
